In [1]:
import pandas as pd
import numpy as np
import json
from pathlib import Path

In [2]:
DATA_DIR = "./data/"
train = pd.read_csv(DATA_DIR + "train.csv")
print("\t --- Train ---")
display(train.head(1))

print("\t --- Client --- ")
client = pd.read_csv(DATA_DIR + "client.csv")
display(client.head(1))

print("\t -- Historic weather ---")
historical_weather = pd.read_csv(DATA_DIR + "historical_weather.csv")
display(historical_weather.head(1))

print("\t --- Forecast weather ---")
forecast_weather = pd.read_csv(DATA_DIR + "forecast_weather.csv")
display(forecast_weather.head(1))

print("\t --- Electricity Prices ---")
electricity = pd.read_csv(DATA_DIR + "electricity_prices.csv")
display(electricity.head(1))

print("\t --- Gas Prices ---")
gas = pd.read_csv(DATA_DIR + "gas_prices.csv")
display(gas.head(1))

print("\t --- Location ---")
location = pd.read_csv(DATA_DIR + "county_lon_lats.csv")
display(location.head(1))

	 --- Train ---


,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0


	 --- Client --- 


,product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
0,1,0,108,952.89,0,2021-09-01,2


	 -- Historic weather ---


...

,forecast_date,euros_per_mwh,origin_date,data_block_id
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1


	 --- Gas Prices ---


,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id
0,2021-09-01,45.23,46.32,2021-08-31,1


	 --- Location ---


,county,longitude,latitude
0,0,24.2,59.1


In [3]:
# load collated dataset
df = pd.read_csv(DATA_DIR+"collated_train.csv")
display(df.head(1))

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,date,...,target_6_days_ago,target_7_days_ago,target_8_days_ago,target_9_days_ago,target_10_days_ago,target_11_days_ago,target_12_days_ago,target_13_days_ago,target_14_days_ago,target_15_days_ago
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,2021-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018352 entries, 0 to 2018351
Data columns (total 71 columns):
 #   Column                                    Dtype  
---  ------                                    -----  
 0   county                                    int64  
 1   is_business                               int64  
 2   product_type                              int64  
 3   target                                    float64
 4   is_consumption                            int64  
 5   datetime                                  object 
 6   data_block_id                             int64  
 7   row_id                                    int64  
 8   prediction_unit_id                        int64  
 9   date                                      object 
 10  year                                      int64  
 11  quarter                                   int64  
 12  month                                     int64  
 13  week                                      int64  
 14  ho

In [12]:
df.select_dtypes(exclude='number').shape[1]#.info()

7

In [18]:
print(f"Numeric  features: #{df.select_dtypes(exclude='object').shape[1]} {df.select_dtypes(exclude='object')}")


Numeric  features: #64          county  is_business  product_type   target  is_consumption  \
0             0            0             1    0.713               0   
1             0            0             1   96.590               1   
2             0            0             2    0.000               0   
3             0            0             2   17.314               1   
4             0            0             3    2.904               0   
...         ...          ...           ...      ...             ...   
2018347      15            1             0  197.233               1   
2018348      15            1             1    0.000               0   
2018349      15            1             1   28.404               1   
2018350      15            1             3    0.000               0   
2018351      15            1             3  196.240               1   

         data_block_id   row_id  prediction_unit_id  year  quarter  ...  \
0                    0        0                  

In [17]:
print(f"Object features: #{df.select_dtypes(exclude='number').shape[1]} {df.select_dtypes(exclude='number')}")

Object features: #7                     datetime        date date_client  \
0        2021-09-01 00:00:00  2021-09-01         NaN   
1        2021-09-01 00:00:00  2021-09-01         NaN   
2        2021-09-01 00:00:00  2021-09-01         NaN   
3        2021-09-01 00:00:00  2021-09-01         NaN   
4        2021-09-01 00:00:00  2021-09-01         NaN   
...                      ...         ...         ...   
2018347  2023-05-31 23:00:00  2023-05-31  2023-05-29   
2018348  2023-05-31 23:00:00  2023-05-31  2023-05-29   
2018349  2023-05-31 23:00:00  2023-05-31  2023-05-29   
2018350  2023-05-31 23:00:00  2023-05-31  2023-05-29   
2018351  2023-05-31 23:00:00  2023-05-31  2023-05-29   

        forecast_date_electricity origin_date_electricity forecast_date_gas  \
0                             NaN                     NaN               NaN   
1                             NaN                     NaN               NaN   
2                             NaN                     NaN             